In [6]:
import os,sys,subprocess,glob,cftime,importlib,pickle,itertools
from datetime import datetime
import xarray as xr
import numpy as np
import pandas as pd
sys.path.append('../')

def import_from(module, name):
    module = __import__(module, fromlist=[name])
    return getattr(module, name)

from ensembles.ensemble_GKLT import ensemble_GKLT,get_weight_for_selection

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:

import argparse
parser = argparse.ArgumentParser()
parser.add_argument("--project_path", type=str)
parser.add_argument("--experiment_identifiers", nargs='+', default=[f"c{i}" for i in range(1,6)] + [f"p{i}" for i in range(1,6)] + ['c1_initial', 'p1_initial'])
parser.add_argument("--overwrite", action='store_true')
command_line_arguments = parser.parse_args()


usage: ipykernel_launcher.py [-h] [--project_path PROJECT_PATH]
                             [--experiment_identifiers EXPERIMENT_IDENTIFIERS [EXPERIMENT_IDENTIFIERS ...]]
                             [--overwrite]
ipykernel_launcher.py: error: unrecognized arguments: --f=/home/u/u290372/.local/share/jupyter/runtime/kernel-v33abaabb5995408a3c68d73928a40f62d8981741d.json


SystemExit: 2

/home/u/u290372/.conda/envs/py_imps/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [21]:
project_path = '../REA_heat_wEU_JJA'
experiment_identifiers = ['c8_wet','c6_dry','c9_dry']

In [22]:

sys.path.append(project_path)
from experiment_configuration.experiment import experiment

for experiment_identifier in experiment_identifiers:
    print(experiment_identifier)

    exp = experiment(import_from(f'experiment_configuration.{experiment_identifier}', 'config'))
    ens = ensemble_GKLT(exp)
    obs = xr.open_mfdataset(f"{exp.dir_work}/REA_output/{exp.product_name}/NCAR/CESM2/{exp.initial_conditions_name}-x{exp.experiment_identifier[1]}/day/atmos/tas-reg/*/*", concat_dim='sim', combine='nested')['tas'].load()

    x = xr.DataArray(
        obs.values.reshape((-1, exp.n_steps, exp.n_days)),
        dims=['sim','step','time'],
        coords=dict(sim=obs.sim, step=np.arange(0,exp.n_steps,1,'int'), time=np.arange(0,exp.n_days,1,'int'))
    )
    time_sum_over_each_step = x.mean('time')

    scores = np.exp(exp.k * time_sum_over_each_step)


    mean_scores = np.array([
                pd.read_table(f"{exp.dir_work}/GKLT/{exp.experiment_name}/book_keeping/step{step}_evaluation.csv", sep=',')['score'].values.mean()
                for step in range(exp.n_steps)
            ])


    weight_from_algo = obs.mean('time').copy() * np.nan
    weight_from_algo[:] = np.array(
        [
            np.prod(mean_scores[:-1]) / (np.prod(scores[i,:-1]) * exp.n_members)
            for i in range(exp.n_members)
        ]
    )


    prob = obs.mean('time').copy() * np.nan
    prob[:] = np.array([np.sum((obs.mean('time') >= a).astype(float) * weight_from_algo) for a in obs.mean('time')]) 


    naming_d = {
        "project": 'REA_output',
        "product": exp.product_name,
        "institute": 'NCAR',
        "model": 'CESM2',
        "experiment" : f"{exp.initial_conditions_name}-x{experiment_identifier[1]}",
        "realm": 'meta',
    }
    out_dir = '/'.join([exp.dir_work] + [v for k,v in naming_d.items()])

    os.makedirs(out_dir, exist_ok=True)
    xr.Dataset({'probability':prob}).to_netcdf(f"{out_dir}/probability_season_{naming_d['experiment']}.nc")
    xr.Dataset({'weight':weight_from_algo}).to_netcdf(f"{out_dir}/weight_season_{naming_d['experiment']}.nc")
    ens.get_weights_uniqueness()
    xr.Dataset({'uniqueness':ens._uniq_seas}).to_netcdf(f"{out_dir}/uniqueness_season_{naming_d['experiment']}.nc")
    xr.Dataset({'uniqueness':ens._uniq_daily}).to_netcdf(f"{out_dir}/uniqueness_day_{naming_d['experiment']}.nc")

c8_wet
c6_dry
c9_dry
